In [ ]:
# turn large NBA_Shots_2004_2024.csv dataset into SQL database

import sqlite3
import pandas as pd
import os

# File paths
csv_file = "/Users/johnquinlan/Downloads/NBA_Shot_Data/NBA_Shots_2004_2024.csv"
db_file = "/Users/johnquinlan/Downloads/NBA_Shot_Data/NBA_Shots.db"

# Connect to SQLite
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Define table name
table_name = "nba_shots"

# Drop table if it exists (optional)
try:
    cursor.execute(f"DROP TABLE IF EXISTS {table_name}")
    print(f"Dropped table {table_name} if it existed.")
except sqlite3.Error as e:
    print(f"Error dropping table: {e}")

# Create the table based on the CSV file's structure
# Read a small chunk of the CSV to get the columns
try:
    sample_chunk = pd.read_csv(csv_file, nrows=5)
    sample_chunk.to_sql(table_name, conn, if_exists='replace', index=False)
    print(f"Table {table_name} created successfully.")
except Exception as e:
    print(f"Error creating table: {e}")

# Read the entire CSV in chunks and insert into SQLite
chunksize = 100_000  # Adjust based on your RAM (100k rows per chunk)
try:
    for chunk in pd.read_csv(csv_file, chunksize=chunksize):
        chunk.to_sql(table_name, conn, if_exists="append", index=False)
        print(f"Inserted {len(chunk)} rows into {table_name}.")
except Exception as e:
    print(f"Error during CSV insertion: {e}")

# Commit and close
conn.commit()
conn.close()

print(f"Database created: {db_file}, table: {table_name}")


In [ ]:
# File path for the database
db_file = "/Users/johnquinlan/Downloads/NBA_Shot_Data/NBA_Shots.db"

# Connect to SQLite
conn = sqlite3.connect(db_file)

# Define the SQL query
query_2 ="""
WITH shots_by_season AS (
    SELECT 
        SEASON_2, 
        SHOT_TYPE,
        SUM(CASE WHEN EVENT_TYPE = 'Made Shot' THEN 1 ELSE 0 END) AS shots_made,
        COUNT(*) AS shots_attempted,
        CASE 
            WHEN COUNT(*) = 0 THEN 0
            ELSE SUM(CASE WHEN EVENT_TYPE = 'Made Shot' THEN 1 ELSE 0 END) * 1.0 / COUNT(*)
        END AS FG_Percentage
    FROM 
        nba_shots 
    GROUP BY 
        SEASON_2, 
        SHOT_TYPE
)
SELECT *,
    CASE 
        WHEN SHOT_TYPE = '2PT Field Goal' THEN FG_Percentage * 2
        WHEN SHOT_TYPE = '3PT Field Goal' THEN FG_Percentage * 3
        ELSE 0
    END AS EV_Shot
FROM shots_by_season;
"""

# Execute the query and fetch the results
try:
    result2 = pd.read_sql_query(query_2, conn)
    print(result2)
except Exception as e:
    print(f"Error during query execution: {e}")

# Close the connection
conn.close()


In [ ]:
import matplotlib.pyplot as plt

# Separate data for plotting
two_pt_shots = result2[result2['SHOT_TYPE'] == '2PT Field Goal']
three_pt_shots = result2[result2['SHOT_TYPE'] == '3PT Field Goal']

# Plotting
plt.figure(figsize=(12, 6))

# Plot 2PT Field Goals
plt.plot(two_pt_shots['SEASON_2'], two_pt_shots['EV_Shot'], color='red', label='2PT Field Goal', marker='o')

# Plot 3PT Field Goals
plt.plot(three_pt_shots['SEASON_2'], three_pt_shots['EV_Shot'], color='blue', label='3PT Field Goal', marker='o')

# Adding titles and labels
plt.title('Expected Value of Shots (EV_Shot) by Season')
plt.xlabel('Season')
plt.ylabel('EV_Shot')
plt.xticks(rotation=45)
plt.legend()
plt.grid()

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Separate data for plotting
two_pt_shots = result2[result2['SHOT_TYPE'] == '2PT Field Goal']
three_pt_shots = result2[result2['SHOT_TYPE'] == '3PT Field Goal']

# Plotting
plt.figure(figsize=(12, 6))

# Plot 2PT Field Goals
plt.plot(two_pt_shots['SEASON_2'], two_pt_shots['FG_Percentage'], color='red', label='2PT Field Goal', marker='o')

# Plot 3PT Field Goals
plt.plot(three_pt_shots['SEASON_2'], three_pt_shots['FG_Percentage'], color='blue', label='3PT Field Goal', marker='o')

# Adding titles and labels
plt.title('Field Goal Percentage by Season')
plt.xlabel('Season')
plt.ylabel('EV_Shot')
plt.xticks(rotation=45)
plt.legend()
plt.grid()

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# File path for the database
db_file = "/Users/johnquinlan/Downloads/NBA_Shot_Data/NBA_Shots.db"

# Connect to SQLite
conn = sqlite3.connect(db_file)

# Define the SQL query
query_3 ="""
WITH adjusted_shots AS (
    SELECT 
        SEASON_2, 
        CASE 
            WHEN BASIC_ZONE IN ('Right Corner 3', 'Left Corner 3') THEN '2PT Field Goal'
            ELSE SHOT_TYPE
        END AS SHOT_TYPE,
        EVENT_TYPE
    FROM 
        nba_shots
),
shots_by_season AS (
    SELECT 
        SEASON_2, 
        SHOT_TYPE,
        SUM(CASE WHEN EVENT_TYPE = 'Made Shot' THEN 1 ELSE 0 END) AS shots_made,
        COUNT(*) AS shots_attempted,
        CASE 
            WHEN COUNT(*) = 0 THEN 0
            ELSE SUM(CASE WHEN EVENT_TYPE = 'Made Shot' THEN 1 ELSE 0 END) * 1.0 / COUNT(*)
        END AS FG_Percentage
    FROM 
        adjusted_shots 
    GROUP BY 
        SEASON_2, 
        SHOT_TYPE
)
SELECT *,
    CASE 
        WHEN SHOT_TYPE = '2PT Field Goal' THEN FG_Percentage * 2
        WHEN SHOT_TYPE = '3PT Field Goal' THEN FG_Percentage * 3
        ELSE 0
    END AS EV_Shot
FROM shots_by_season;
"""

# Execute the query and fetch the results
try:
    result3 = pd.read_sql_query(query_3, conn)
    print(result3)
except Exception as e:
    print(f"Error during query execution: {e}")

# Close the connection
conn.close()


In [ ]:
# File path for the database
db_file = "/Users/johnquinlan/Downloads/NBA_Shot_Data/NBA_Shots.db"

# Connect to SQLite
conn = sqlite3.connect(db_file)

# Define the SQL query
query_4 ="""
WITH corner_3_shots_23_24 AS (
    SELECT 
        PLAYER_NAME, 
        SUM(CASE WHEN BASIC_ZONE IN ('Right Corner 3', 'Left Corner 3') AND EVENT_TYPE = 'Made Shot' THEN 1 ELSE 0 END) AS corner_3_shots_made,
        SUM(CASE WHEN BASIC_ZONE IN ('Right Corner 3', 'Left Corner 3') THEN 1 ELSE 0 END) AS corner_3_shots_attempted,
        SUM(CASE WHEN EVENT_TYPE = 'Made Shot' THEN 1 ELSE 0 END) AS overall_shots_made,
        COUNT(*) AS overall_shots_attempted
    FROM 
        nba_shots 
    WHERE SEASON_2 = '2023-24'
    GROUP BY 
        PLAYER_NAME
)
SELECT 
    PLAYER_NAME,
    corner_3_shots_made,
    corner_3_shots_attempted,
    CASE 
        WHEN corner_3_shots_attempted = 0 THEN 0
        ELSE corner_3_shots_made * 1.0 / corner_3_shots_attempted
    END AS corner_3_percentage,
    overall_shots_made,
    overall_shots_attempted,
    CASE 
        WHEN overall_shots_attempted = 0 THEN 0
        ELSE overall_shots_made * 1.0 / overall_shots_attempted
    END AS overall_percentage,
        CASE 
        WHEN overall_shots_attempted = 0 THEN 0
        ELSE corner_3_shots_attempted * 1.0 / overall_shots_attempted
    END AS corner_3_percentage_of_shots
FROM corner_3_shots_23_24
WHERE overall_shots_attempted > 100
ORDER BY corner_3_percentage_of_shots DESC;
"""

# Execute the query and fetch the results
try:
    result4 = pd.read_sql_query(query_4, conn)
    print(result4)
except Exception as e:
    print(f"Error during query execution: {e}")

# Close the connection
conn.close()

In [ ]:
# File path for the database
db_file = "/Users/johnquinlan/Downloads/NBA_Shot_Data/NBA_Shots.db"

# Connect to SQLite
conn = sqlite3.connect(db_file)

# Define the SQL query
query_5 ="""
WITH shot_totals AS (
    SELECT  
        SEASON_2,
        SUM(CASE WHEN BASIC_ZONE IN ('Right Corner 3', 'Left Corner 3') AND EVENT_TYPE = 'Made Shot' THEN 1 ELSE 0 END) AS corner_3_shots_made,
        SUM(CASE WHEN BASIC_ZONE IN ('Right Corner 3', 'Left Corner 3') THEN 1 ELSE 0 END) AS corner_3_shots_attempted,
        SUM(CASE WHEN EVENT_TYPE = 'Made Shot' THEN 1 ELSE 0 END) AS overall_shots_made,
        COUNT(*) AS overall_shots_attempted,
        SUM(CASE WHEN SHOT_TYPE = '3PT Field Goal' AND EVENT_TYPE = 'Made Shot' THEN 1 ELSE 0 END) AS three_pt_shots_made,
        SUM(CASE WHEN SHOT_TYPE = '3PT Field Goal' THEN 1 ELSE 0 END) AS three_pt_shots_attempted,
        SUM(CASE WHEN SHOT_TYPE = '2PT Field Goal' AND EVENT_TYPE = 'Made Shot' THEN 1 ELSE 0 END) AS two_pt_shots_made,
        SUM(CASE WHEN SHOT_TYPE = '2PT Field Goal' THEN 1 ELSE 0 END) AS two_pt_shots_attempted
    FROM 
        nba_shots 
    GROUP BY 
        SEASON_2
)
SELECT *, 
    corner_3_shots_made * 1.0 / corner_3_shots_attempted AS corner_3_fg_percentage,
    three_pt_shots_made * 1.0 / three_pt_shots_attempted AS three_pt_fg_percentage,
    two_pt_shots_made * 1.0 / two_pt_shots_attempted AS two_pt_fg_percentage
FROM shot_totals;
"""
# Execute the query and fetch the results
try:
    result5 = pd.read_sql_query(query_5, conn)
    print(result5)
except Exception as e:
    print(f"Error during query execution: {e}")

# Close the connection
conn.close()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming result5 is already your DataFrame with the necessary data

# Graph 1: Line Graph for Shot Attempts
plt.figure(figsize=(12, 6))
plt.plot(result5['SEASON_2'], result5['three_pt_shots_attempted'], label='3 PT Attempts', marker='o')
plt.plot(result5['SEASON_2'], result5['two_pt_shots_attempted'], label='2 PT Attempts', marker='o')
plt.plot(result5['SEASON_2'], result5['corner_3_shots_attempted'], label='Corner 3 Attempts', marker='o')
plt.title('Shot Attempts by Season')
plt.xlabel('Seasons')
plt.ylabel('Shots Attempted')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Graph 2: Line Graph for Field Goal Percentages
plt.figure(figsize=(12, 6))
plt.plot(result5['SEASON_2'], result5['three_pt_fg_percentage'], label='3 PT FG Percentage', marker='o')
plt.plot(result5['SEASON_2'], result5['two_pt_fg_percentage'], label='2 PT FG Percentage', marker='o')
plt.plot(result5['SEASON_2'], result5['corner_3_fg_percentage'], label='Corner 3 FG Percentage', marker='o')
plt.title('Field Goal Percentages by Season')
plt.xlabel('Seasons')
plt.ylabel('Field Goal Percentage')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# File path for the database
db_file = "/Users/johnquinlan/Downloads/NBA_Shot_Data/NBA_Shots.db"

# Connect to SQLite
conn = sqlite3.connect(db_file)

# Define the SQL query
query_6 ="""
WITH shot_totals AS (
    SELECT  
        SEASON_2,
        BASIC_ZONE,
        SUM(CASE WHEN EVENT_TYPE = 'Made Shot' THEN 1 ELSE 0 END) AS shots_made,
        COUNT(*) AS shots_attempted
    FROM 
        nba_shots 
    GROUP BY 
        BASIC_ZONE, SEASON_2
)
SELECT *, 
    shots_made * 1.0 / NULLIF(shots_attempted, 0) AS fg_percentage
FROM shot_totals;
"""
# Execute the query and fetch the results
try:
    result6 = pd.read_sql_query(query_6, conn)
    print(result6)
except Exception as e:
    print(f"Error during query execution: {e}")


# Close the connection
conn.close()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming result6 is your DataFrame with columns: SEASON_2, BASIC_ZONE, and shots_attempted
# First, pivot the DataFrame to have BASIC_ZONE types as columns
pivot_df = result6.pivot(index='SEASON_2', columns='BASIC_ZONE', values='shots_attempted').fillna(0)

# Plotting
plt.figure(figsize=(12, 6))
for column in pivot_df.columns:
    plt.plot(pivot_df.index, pivot_df[column], label=column)

plt.title('Shots Attempted by BASIC_ZONE Over Seasons')
plt.xlabel('Seasons')
plt.ylabel('Shots Attempted')
plt.xticks(rotation=45)
plt.legend(title='BASIC_ZONE', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


# First, pivot the DataFrame to have BASIC_ZONE types as columns for fg_percentage
pivot_fg_df = result6.pivot(index='SEASON_2', columns='BASIC_ZONE', values='fg_percentage').fillna(0)

# Plotting
plt.figure(figsize=(12, 6))
for column in pivot_fg_df.columns:
    plt.plot(pivot_fg_df.index, pivot_fg_df[column], label=column)

plt.title('Field Goal Percentages by BASIC_ZONE Over Seasons')
plt.xlabel('Seasons')
plt.ylabel('Field Goal Percentage')
plt.xticks(rotation=45)
plt.legend(title='BASIC_ZONE', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming result6 is your DataFrame with columns: SEASON_2, BASIC_ZONE, shots_attempted, and fg_percentage

# First, pivot the DataFrame to have BASIC_ZONE types as columns
pivot_df = result6.pivot(index='SEASON_2', columns='BASIC_ZONE', values='shots_attempted').fillna(0)

# Plotting and saving the first graph
plt.figure(figsize=(12, 6))
for column in pivot_df.columns:
    plt.plot(pivot_df.index, pivot_df[column], label=column)

plt.title('Shots Attempted by BASIC_ZONE Over Seasons')
plt.xlabel('Seasons')
plt.ylabel('Shots Attempted')
plt.xticks(rotation=45)
plt.legend(title='BASIC_ZONE', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

# Save and close the first plot
plt.savefig('shots_attempted_over_seasons.png', dpi=300)
plt.close()

# Pivot the DataFrame for field goal percentages
pivot_fg_df = result6.pivot(index='SEASON_2', columns='BASIC_ZONE', values='fg_percentage').fillna(0)

# Plotting and saving the second graph
plt.figure(figsize=(12, 6))
for column in pivot_fg_df.columns:
    plt.plot(pivot_fg_df.index, pivot_fg_df[column], label=column)

plt.title('Field Goal Percentages by BASIC_ZONE Over Seasons')
plt.xlabel('Seasons')
plt.ylabel('Field Goal Percentage')
plt.xticks(rotation=45)
plt.legend(title='BASIC_ZONE', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

# Save and close the second plot
plt.savefig('fg_percentage_over_seasons.png', dpi=300)
plt.close()


In [ ]:
# File path for the database
db_file = "/Users/johnquinlan/Downloads/NBA_Shot_Data/NBA_Shots.db"

# Connect to SQLite
conn = sqlite3.connect(db_file)

# Define the SQL query
query_7 ="""
WITH categorized_shots AS (
    SELECT 
        SEASON_2,
        CASE 
            WHEN BASIC_ZONE IN ('Above the Break 3', 'Right Corner 3', 'Left Corner 3') THEN 'Three Pointer'
            WHEN BASIC_ZONE = 'Mid-Range' THEN 'Mid-Range'
            WHEN BASIC_ZONE IN ('Restricted Area', 'In the Paint (NON-RA)') THEN 'Paint Shots'
            ELSE NULL
        END AS SHOT_CATEGORY,
        EVENT_TYPE
    FROM 
        nba_shots
    WHERE 
        BASIC_ZONE <> 'Back Court'  -- Exclude Back Court shots
),
shots_summary AS (
    SELECT 
        SEASON_2,
        SHOT_CATEGORY,
        SUM(CASE WHEN EVENT_TYPE = 'Made Shot' THEN 1 ELSE 0 END) AS shots_made,
        COUNT(*) AS shots_attempted,
        CASE 
            WHEN COUNT(*) = 0 THEN 0
            ELSE SUM(CASE WHEN EVENT_TYPE = 'Made Shot' THEN 1 ELSE 0 END) * 1.0 / COUNT(*)
        END AS FG_Percentage,
        CASE 
            WHEN SHOT_CATEGORY = 'Three Pointer' THEN 
                CASE 
                    WHEN COUNT(*) = 0 THEN 0
                    ELSE SUM(CASE WHEN EVENT_TYPE = 'Made Shot' THEN 1 ELSE 0 END) * 1.0 / COUNT(*) * 3  -- 3 points for a three-pointer
                END
            WHEN SHOT_CATEGORY IN ('Mid-Range', 'Paint Shots') THEN 
                CASE 
                    WHEN COUNT(*) = 0 THEN 0
                    ELSE SUM(CASE WHEN EVENT_TYPE = 'Made Shot' THEN 1 ELSE 0 END) * 1.0 / COUNT(*) * 2  -- 2 points for mid-range and paint shots
                END
            ELSE 0
        END AS EV_Shot
    FROM 
        categorized_shots 
    WHERE 
        SHOT_CATEGORY IS NOT NULL  -- Only include valid shot categories
    GROUP BY 
        SEASON_2, 
        SHOT_CATEGORY
)
SELECT * FROM shots_summary
;

"""
# Execute the query and fetch the results
try:
    result7 = pd.read_sql_query(query_7, conn)
    print(result7)
except Exception as e:
    print(f"Error during query execution: {e}")


# Close the connection
conn.close()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# First Graph: Shots Attempted by Season and Shot Category
plt.figure(figsize=(12, 6))
for category in result7['SHOT_CATEGORY'].unique():
    subset = result7[result7['SHOT_CATEGORY'] == category]
    plt.plot(subset['SEASON_2'], subset['shots_attempted'], marker='o', label=category)

plt.title('Shots Attempted by Season and Shot Category')
plt.xlabel('Season')
plt.ylabel('Shots Attempted')
plt.xticks(rotation=45)
plt.legend(title='Shot Category')
plt.grid()
plt.tight_layout()
plt.show()

# Second Graph: FG Percentage by Season and Shot Category
plt.figure(figsize=(12, 6))
for category in result7['SHOT_CATEGORY'].unique():
    subset = result7[result7['SHOT_CATEGORY'] == category]
    plt.plot(subset['SEASON_2'], subset['FG_Percentage'], marker='o', label=category)

plt.title('Field Goal Percentage by Season and Shot Category')
plt.xlabel('Season')
plt.ylabel('Field Goal Percentage')
plt.xticks(rotation=45)
plt.legend(title='Shot Category')
plt.grid()
plt.tight_layout()
plt.show()

# Third Graph: Expected Value by Season and Shot Category
plt.figure(figsize=(12, 6))
for category in result7['SHOT_CATEGORY'].unique():
    subset = result7[result7['SHOT_CATEGORY'] == category]
    plt.plot(subset['SEASON_2'], subset['EV_Shot'], marker='o', label=category)

plt.title('Expected Value by Season and Shot Category')
plt.xlabel('Season')
plt.ylabel('Expected Value (EV)')
plt.xticks(rotation=45)
plt.legend(title='Shot Category')
plt.grid()
plt.tight_layout()
plt.show()




In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Define the folder path for saving images
folder_path = '/Users/johnquinlan/Downloads/NBA_Shot_Data/images/'

# First Graph: Shots Attempted by Season and Shot Category
plt.figure(figsize=(12, 6))
for category in result7['SHOT_CATEGORY'].unique():
    subset = result7[result7['SHOT_CATEGORY'] == category]
    plt.plot(subset['SEASON_2'], subset['shots_attempted'], marker='o', label=category)

plt.title('Shots Attempted by Season and Shot Category')
plt.xlabel('Season')
plt.ylabel('Shots Attempted')
plt.xticks(rotation=45)
plt.legend(title='Shot Category')
plt.grid()
plt.tight_layout()
plt.savefig(folder_path + 'Shot_Attempts_By_Major_Category.png')  # Save the figure
plt.close()  # Close the figure

# Second Graph: FG Percentage by Season and Shot Category
plt.figure(figsize=(12, 6))
for category in result7['SHOT_CATEGORY'].unique():
    subset = result7[result7['SHOT_CATEGORY'] == category]
    plt.plot(subset['SEASON_2'], subset['FG_Percentage'], marker='o', label=category)

plt.title('Field Goal Percentage by Season and Shot Category')
plt.xlabel('Season')
plt.ylabel('Field Goal Percentage')
plt.xticks(rotation=45)
plt.legend(title='Shot Category')
plt.grid()
plt.tight_layout()
plt.savefig(folder_path + 'FG_Percentage_By_Major_Category.png')  # Save the figure
plt.close()  # Close the figure

# Third Graph: Expected Value by Season and Shot Category
plt.figure(figsize=(12, 6))
for category in result7['SHOT_CATEGORY'].unique():
    subset = result7[result7['SHOT_CATEGORY'] == category]
    plt.plot(subset['SEASON_2'], subset['EV_Shot'], marker='o', label=category)

plt.title('Expected Value by Season and Shot Category')
plt.xlabel('Season')
plt.ylabel('Expected Value (EV)')
plt.xticks(rotation=45)
plt.legend(title='Shot Category')
plt.grid()
plt.tight_layout()
plt.savefig(folder_path + 'EV_by_Major_Category.png')  # Save the figure
plt.close()  # Close the figure


In [ ]:
import sqlite3
import pandas as pd

# File path for the database
db_file = "/Users/johnquinlan/Downloads/NBA_Shot_Data/NBA_Shots.db"

# Connect to SQLite
conn = sqlite3.connect(db_file)

# Define the SQL query
query_8 = """  
WITH corner_3_shots AS (
    SELECT 
        SEASON_2,
        PLAYER_NAME, 
        SUM(CASE WHEN BASIC_ZONE IN ('Right Corner 3', 'Left Corner 3') AND EVENT_TYPE = 'Made Shot' THEN 1 ELSE 0 END) AS corner_3_shots_made,
        SUM(CASE WHEN BASIC_ZONE IN ('Right Corner 3', 'Left Corner 3') THEN 1 ELSE 0 END) AS corner_3_shots_attempted,
        SUM(CASE WHEN EVENT_TYPE = 'Made Shot' THEN 1 ELSE 0 END) AS overall_shots_made,
        COUNT(*) AS overall_shots_attempted
    FROM nba_shots 
    GROUP BY SEASON_2, PLAYER_NAME
),
ranked_players AS (
    SELECT 
        SEASON_2,
        PLAYER_NAME,
        corner_3_shots_made,
        corner_3_shots_attempted,
        CASE 
            WHEN corner_3_shots_attempted = 0 THEN 0
            ELSE corner_3_shots_made * 1.0 / corner_3_shots_attempted
        END AS corner_3_percentage,
        overall_shots_made,
        overall_shots_attempted,
        CASE 
            WHEN overall_shots_attempted = 0 THEN 0
            ELSE overall_shots_made * 1.0 / overall_shots_attempted
        END AS overall_percentage,
        CASE 
            WHEN overall_shots_attempted = 0 THEN 0
            ELSE corner_3_shots_attempted * 1.0 / overall_shots_attempted
        END AS corner_3_percentage_of_shots,
        ROW_NUMBER() OVER (PARTITION BY SEASON_2 ORDER BY (corner_3_shots_attempted * 1.0 / overall_shots_attempted) DESC) AS rank
    FROM corner_3_shots
    WHERE overall_shots_attempted > 100
)
SELECT 
    SEASON_2,
    AVG(corner_3_percentage_of_shots) AS avg_corner_3_percentage_of_shots_top_60
FROM ranked_players
WHERE rank <= 60
GROUP BY SEASON_2
ORDER BY SEASON_2;
"""

# Execute the query and fetch the results
try:
    result8 = pd.read_sql_query(query_8, conn)
    print(result8)
except Exception as e:
    print(f"Error during query execution: {e}")

# Close the connection
conn.close()


In [ ]:
import sqlite3
import pandas as pd

# File path for the database
db_file = "/Users/johnquinlan/Downloads/NBA_Shot_Data/NBA_Shots.db"

# Connect to SQLite
conn = sqlite3.connect(db_file)

# Define the SQL query
query_three = """  
WITH three_shots AS (
    SELECT 
        SEASON_2,
        PLAYER_NAME, 
        SUM(CASE WHEN BASIC_ZONE IN ('Right Corner 3', 'Left Corner 3', 'Above the Break 3') AND EVENT_TYPE = 'Made Shot' THEN 1 ELSE 0 END) AS three_shots_made,
        SUM(CASE WHEN BASIC_ZONE IN ('Right Corner 3', 'Left Corner 3', 'Above the Break 3') THEN 1 ELSE 0 END) AS three_shots_attempted,
        SUM(CASE WHEN EVENT_TYPE = 'Made Shot' THEN 1 ELSE 0 END) AS overall_shots_made,
        COUNT(*) AS overall_shots_attempted
    FROM nba_shots 
    GROUP BY SEASON_2, PLAYER_NAME
),
ranked_players AS (
    SELECT 
        SEASON_2,
        PLAYER_NAME,
        three_shots_made,
        three_shots_attempted,
        CASE 
            WHEN three_shots_attempted = 0 THEN 0
            ELSE three_shots_made * 1.0 / three_shots_attempted
        END AS three_percentage,
        overall_shots_made,
        overall_shots_attempted,
        CASE 
            WHEN overall_shots_attempted = 0 THEN 0
            ELSE overall_shots_made * 1.0 / overall_shots_attempted
        END AS overall_percentage,
        CASE 
            WHEN overall_shots_attempted = 0 THEN 0
            ELSE three_shots_attempted * 1.0 / overall_shots_attempted
        END AS three_percentage_of_shots,
        ROW_NUMBER() OVER (PARTITION BY SEASON_2 ORDER BY (three_shots_attempted * 1.0 / overall_shots_attempted) DESC) AS rank
    FROM three_shots
    WHERE overall_shots_attempted > 100
)
SELECT 
    SEASON_2,
    AVG(three_percentage_of_shots) AS avg_three_percentage_of_shots_league,
    AVG(CASE WHEN rank <= 60 THEN three_percentage_of_shots ELSE NULL END) AS avg_three_percentage_of_shots_top_60
FROM ranked_players
GROUP BY SEASON_2
ORDER BY SEASON_2;

"""

# Execute the query and fetch the results
try:
    result_three = pd.read_sql_query(query_three, conn)
    print(result_three)
except Exception as e:
    print(f"Error during query execution: {e}")

# Close the connection
conn.close()


In [ ]:
import matplotlib.pyplot as plt

# Assuming result_three is the DataFrame with SQL query results
plt.figure(figsize=(10, 5))

# Plot league-wide average
plt.plot(result_three['SEASON_2'], result_three['avg_three_percentage_of_shots_league'], 
         label='League Average', marker='o', linestyle='-')

# Plot top 60 players' average
plt.plot(result_three['SEASON_2'], result_three['avg_three_percentage_of_shots_top_60'], 
         label='Top 60 Players', marker='s', linestyle='--')

# Labels and title
plt.xlabel('Seasons')
plt.ylabel('3PT Attempt of Total Shots Percentage')
plt.title('3PT Attempt of Total Shots Percentage Over Seasons')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)

# Save and show plot
plt.savefig('three_point_percentage_trends.png', dpi=300)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Ensure SEASON_2 is sorted correctly
result8 = result8.sort_values(by='SEASON_2')

# Set figure size
plt.figure(figsize=(12, 6))

# Plot the data
plt.plot(result8['SEASON_2'], result8['avg_corner_3_percentage_of_shots_top_60'], marker='o', linestyle='-', color='b')

# Labels and title
plt.xlabel('Season')
plt.ylabel('Avg % Corner 3 Shots of Total Shots')
plt.title('Trend of Corner 3 Shot Percentage of Total Shots (Top 60 Players Corner Shooters)')

# Rotate x-axis labels for better readability
plt.xticks(result8['SEASON_2'], rotation=45, ha='right', fontsize=9)

# Ensure the full x-axis range is displayed
plt.xlim(result8['SEASON_2'].min(), result8['SEASON_2'].max())

# Grid for clarity
plt.grid(True, linestyle='--', alpha=0.7)

# Show plot
plt.savefig(folder_path + 'Corner_3_specialist_trend.png')  # Save the figure
plt.close()



In [ ]:
import sqlite3
import pandas as pd

# File path for the database
db_file = "/Users/johnquinlan/Downloads/NBA_Shot_Data/NBA_Shots.db"

# Connect to SQLite
conn = sqlite3.connect(db_file)

# Define SQL queries for different seasons
query_23_24_shots = "SELECT * FROM nba_shots WHERE SEASON_2 = '2023-24'"
query_13_14_shots = "SELECT * FROM nba_shots WHERE SEASON_2 = '2013-14'"
query_03_04_shots = "SELECT * FROM nba_shots WHERE SEASON_2 = '2003-04'"

# Execute queries and store results
try:
    result9 = pd.read_sql_query(query_23_24_shots, conn)
    result10 = pd.read_sql_query(query_13_14_shots, conn)
    result11 = pd.read_sql_query(query_03_04_shots, conn)

    # Print confirmation
    print("Data successfully retrieved for all three seasons.")

except Exception as e:
    print(f"Error during query execution: {e}")

# Close the connection
conn.close()


In [ ]:
import plotly.express as px

# Function to create and return a heatmap for a given dataset
def create_heatmap(df, season):
    filtered_df = df[df['LOC_Y'] <= 50]  # Filter shots within Y ≤ 50
    fig = px.density_heatmap(
        filtered_df, 
        x='LOC_X', 
        y='LOC_Y', 
        z='LOC_X', 
        histfunc='count', 
        color_continuous_scale='YlGn',
        range_color=[0, 500]  # Set the color range
    )

    # Update layout
    fig.update_layout(
        title=f'Heatmap of NBA Shot Locations ({season} Season)',
        xaxis_title='Location X',
        yaxis_title='Location Y',
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=False)
    )

    return fig

# Create heatmaps for all seasons
fig9 = create_heatmap(result9, "2023-24")
fig10 = create_heatmap(result10, "2013-14")
fig11 = create_heatmap(result11, "2003-04")

# Show the heatmaps
fig9.show()
fig10.show()
fig11.show()





In [ ]:
import sqlite3
import pandas as pd

# File path for the database
db_file = "/Users/johnquinlan/Downloads/NBA_Shot_Data/NBA_Shots.db"

# Connect to SQLite
conn = sqlite3.connect(db_file)

# Define SQL queries for different seasons
query_giannis_23_24_shots = "SELECT * FROM nba_shots WHERE SEASON_2 = '2023-24' AND PLAYER_NAME = 'Giannis Antetokounmpo'"
query_giannis_13_14_shots = "SELECT * FROM nba_shots WHERE SEASON_2 = '2013-14' AND PLAYER_NAME = 'Giannis Antetokounmpo'"

# Execute queries and store results
try:
    result_giannis_13 = pd.read_sql_query(query_giannis_23_24_shots, conn)
    result_giannis_23 = pd.read_sql_query(query_giannis_13_14_shots, conn)

    # Print confirmation
    print("Data successfully retrieved for both seasons.")

except Exception as e:
    print(f"Error during query execution: {e}")

# Close the connection
conn.close()

In [ ]:
import plotly.express as px

def create_heatmap(df, season):
    filtered_df = df[df['LOC_Y'] <= 50]  # Filter shots within Y ≤ 50
    fig = px.density_heatmap(
        filtered_df, 
        x='LOC_X', 
        y='LOC_Y', 
        z='LOC_X', 
        histfunc='count', 
        color_continuous_scale='YlGn',
        range_color=[0, 25]  # Set the color range
    )

    # Update layout
    fig.update_layout(
        title=f'Heatmap of NBA Shot Locations ({season} Season)',
        xaxis_title='Location X',
        yaxis_title='Location Y',
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=False)
    )

    return fig

# Create heatmaps for all seasons
fig_giannis_13 = create_heatmap(result_giannis_13, "Giannis 2023-24")
fig_giannis_23 = create_heatmap(result_giannis_23, "Giannis 2013-14")


# Show the heatmaps
fig_giannis_13.show()
fig_giannis_23.show()



In [ ]:
import sqlite3
import pandas as pd

# File path for the database
db_file = "/Users/johnquinlan/Downloads/NBA_Shot_Data/NBA_Shots.db"

# Connect to SQLite
conn = sqlite3.connect(db_file)

# Define the SQL query
query_lobs = """  
SELECT 
    SEASON_2, 
    SUM(CASE WHEN ACTION_TYPE = 'Alley Oop Dunk Shot' AND EVENT_TYPE = 'Made Shot' THEN 1 ELSE 0 END) AS Lobs
FROM nba_shots 
GROUP BY SEASON_2;

"""

# Execute the query and fetch the results
try:
    result_lobs = pd.read_sql_query(query_lobs, conn)
    print(result_lobs)
except Exception as e:
    print(f"Error during query execution: {e}")

# Close the connection
conn.close()

In [ ]:
import sqlite3
import pandas as pd

# File path for the database
db_file = "/Users/johnquinlan/Downloads/NBA_Shot_Data/NBA_Shots.db"

# Connect to SQLite
conn = sqlite3.connect(db_file)

# Define the SQL query
query_clutch_23_24 = """  
WITH clutch_shots_24 AS (
    SELECT 
        PLAYER_NAME, 
        SUM(CASE WHEN EVENT_TYPE = 'Made Shot' THEN 1 ELSE 0 END) AS Shots_Made,
        COUNT(*) AS Shots_Attempted
    FROM nba_shots 
    WHERE QUARTER = 4 AND MINS_LEFT IN ('0', '1') AND SEASON_2 = '2023-24'
    GROUP BY PLAYER_NAME
)
SELECT *,
    ROUND(Shots_Made * 1.0 / NULLIF(Shots_Attempted, 0), 3) AS FG_Percentage
FROM clutch_shots_24 
WHERE Shots_Attempted > 50
ORDER BY FG_Percentage DESC
LIMIT 20;

"""

# Execute the query and fetch the results
try:
    result_clutch_24 = pd.read_sql_query(query_clutch_23_24, conn)
    print(result_clutch_24)
except Exception as e:
    print(f"Error during query execution: {e}")

# Close the connection
conn.close()